In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
top_60_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(60)
top_60_coins_df = top_60_coins_df.reset_index(drop=True)
top_60_coins = top_60_coins_df.get('name')

In [3]:
from constants.coin_names import CoinNames
from settings import DATA_DIR
import pandas as pd
import os
dfs = {}
no_data = []
# for coin in CoinNames.__members__:
for coin in top_60_coins:
    p = DATA_DIR / f'{coin}.csv'
    if os.path.exists(p):
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        dfs[coin] = df
    else:
        no_data.append(coin)
else:
    os.sys.stderr.write(f'No data for {no_data}')

No data for []

In [4]:
dfs["SOL"].iloc[-1, :]["tic"]

1712793600000

In [5]:
from collections import Counter


def get_majority_shape(dfs):
    # Get the shapes of all DataFrames
    shapes = [df.shape for df in dfs.values()]
    
    # Count occurrences of each shape
    shape_counts = Counter(shapes)
    
    # Display votes for each shape
    for shape, count in shape_counts.items():
        print(f"Shape: {shape}, Votes: {count}")
    
    # Find the most common shape
    majority_shape, majority_votes = shape_counts.most_common(1)[0]
    
    return majority_shape

def get_majority_start_end(dfs):
    # Get the shapes of all DataFrames
    start_end = [(df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"] ) for df in dfs.values()]
    
    # Count occurrences of each shape
    start_end_counts = Counter(start_end)
    
    # Display votes for each shape
    for se, count in start_end_counts.items():
        print(f"Start End: {se}, Votes: {count}")
    
    # Find the most common shape
    majority_se, majority_votes = start_end_counts.most_common(1)[0]
    
    return majority_se

major_shape = get_majority_shape(dfs)
major_se = get_majority_start_end(dfs)

Shape: (2881, 11), Votes: 58
Shape: (810, 11), Votes: 1
Shape: (203, 11), Votes: 1
Start End: (1710201600000, 1712793600000), Votes: 58
Start End: (1712065500000, 1712793600000), Votes: 1
Start End: (1712611800000, 1712793600000), Votes: 1


In [6]:
def filter_dfs(dfs, shape, start_end):
    valid_dfs = {}
    for coin, df in dfs.items():
        if df.shape != shape:
            print(f"Shape mismatch for {coin}. Expected {shape}, got {df.shape}")
            continue
        elif (df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"]) != start_end:
            print(f"Start/End mismatch for {coin}. Expected {start_end}, got {df.iloc[0, :]['tic'], df.iloc[-1, :]['tic']}")
            continue
        else:
            valid_dfs[coin] = df

    return valid_dfs


valid_dfs = filter_dfs(dfs, major_shape, major_se)

Shape mismatch for ENA. Expected (2881, 11), got (810, 11)
Shape mismatch for TNSR. Expected (2881, 11), got (203, 11)


In [7]:
df = valid_dfs["SOL"]

df.head()

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,
SOL,2024-03-12-00:00:00,2024-03-12-00:14:59,15m,339,148.23,149.70,148.18,149.11,10850.12,1710201600000,1710202499999
SOL,2024-03-12-00:15:00,2024-03-12-00:29:59,15m,188,148.94,148.94,148.09,148.22,4560.40,1710202500000,1710203399999
SOL,2024-03-12-00:30:00,2024-03-12-00:44:59,15m,184,148.70,149.42,148.66,148.94,8424.08,1710203400000,1710204299999
SOL,2024-03-12-00:45:00,2024-03-12-00:59:59,15m,252,147.43,148.69,147.40,148.68,6911.43,1710204300000,1710205199999
SOL,2024-03-12-01:00:00,2024-03-12-01:14:59,15m,318,148.21,148.21,147.35,147.41,8708.57,1710205200000,1710206099999


In [8]:
_df = pd.concat(valid_dfs.values())
_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167098 entries, BTC to W
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   start_time        167098 non-null  object 
 1   end_time          167098 non-null  object 
 2   interval          167098 non-null  object 
 3   number_of_trades  167098 non-null  int64  
 4   close             167098 non-null  float64
 5   high              167098 non-null  float64
 6   low               167098 non-null  float64
 7   open              167098 non-null  float64
 8   volume            167098 non-null  float64
 9   tic               167098 non-null  int64  
 10  toc               167098 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 15.3+ MB


In [9]:
_df.head()

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,
BTC,2024-03-12-00:00:00,2024-03-12-00:14:59,15m,740,72253.0,72389.0,72129.0,72155.0,73.59512,1710201600000,1710202499999
BTC,2024-03-12-00:15:00,2024-03-12-00:29:59,15m,544,72189.0,72360.0,72177.0,72254.0,33.93085,1710202500000,1710203399999
BTC,2024-03-12-00:30:00,2024-03-12-00:44:59,15m,441,72351.0,72411.0,72182.0,72188.0,38.35377,1710203400000,1710204299999
BTC,2024-03-12-00:45:00,2024-03-12-00:59:59,15m,666,72077.0,72351.0,72063.0,72350.0,39.48791,1710204300000,1710205199999
BTC,2024-03-12-01:00:00,2024-03-12-01:14:59,15m,735,72329.0,72360.0,71970.0,72076.0,74.58844,1710205200000,1710206099999


In [10]:
# import numpy as np


df = _df
# ############ pct_change
# df["pct_change"] = df.groupby(level=0)["close"].pct_change() * 100

# ############ accumulated_pct_change
# df['multiplier'] = 1 + (df['pct_change'] / 100)
# df['accumulated_multiplier'] = df.groupby(level=0)['multiplier'].cumprod()
# df['accumulated_pct_change'] = (df['accumulated_multiplier'] - 1) * 100
# df.drop(columns=['multiplier', 'accumulated_multiplier'], inplace=True)

# ############ rolling_accumulated_pct_change
# df['start_time'] = pd.to_datetime(df['start_time'])
# df['multiplier'] = 1 + (df['pct_change'] / 100)
# window_size = 7 * 24 * 4  # Adjust this based on your exact data frequency
# df['rolling_accumulated_multiplier'] = df.groupby(level=0)['multiplier'].transform(
#     lambda x: x.rolling(window=window_size, min_periods=1).apply(np.prod)
# )
# df['rolling_accumulated_pct_change'] = (df['rolling_accumulated_multiplier'] - 1) * 100
# df.drop(columns=['multiplier', 'rolling_accumulated_multiplier'], inplace=True)



# df

In [11]:
df.to_csv(DATA_DIR / "all.csv", index=True)